# Balances

<div class="alert alert-danger">
  <strong>Warning!</strong> This notebook contains a fake portfolio <i>(automatically generated)</i> and <b>does not</b> represent my own, it merely serves as an example.
</div>


In [ ]:
from datetime import datetime
print(f"Last executed on {datetime.now()}")

### Binance

Uses API and gets both wallet and Binance Earn investments.

### BlockFi

Uses `csv` files downloaded from https://app.blockfi.com/settings/reports

### Nexo.io

Uses `csv` files downloaded from https://platform.nexo.io/transactions

### Exodus

Uses `csv` files which can be extracted using:

Exodus Desktop application → *Exodus* in menu bar → *Developer* → *Export All Transactions*


TODO: get old Exodus wallet

### Trust Wallet

Uses an BCSScan.com API query for all BEP20 tokens.

In [ ]:
import net_worth_tracker as nwt

balances_binance = nwt.binance.get_binance_balances()
balances_blockfi = nwt.blockfi.get_blockfi_balances()
balances_nexo = nwt.nexo.get_nexo_balances()
balances_exodus = nwt.exodus.get_exodus()
balances_bep20 = nwt.binance_smart_chain.get_bep20_balances()

yieldwatch = nwt.binance_smart_chain.scrape_yieldwatch(timeout=30)
balances_bsc = nwt.binance_smart_chain.yieldwatch_to_balances(yieldwatch)

In [ ]:
balances_degiro = nwt.degiro.get_degiro_balances()

brand_new_day = nwt.brand_new_day.scrape_brand_new_day()
balances_brand_new_day = nwt.brand_new_day.get_balances(brand_new_day)

In [ ]:
balances = {
    "binance": balances_binance,
    "blockfi": balances_blockfi,
    "nexo": balances_nexo,
    "exodus": balances_exodus,
    "bep20": balances_bep20,
    "bsc": balances_bsc,
    "degiro": balances_degiro,
    "brand_new_day": balances_brand_new_day,
}

# Add crypto prices from CoinGecko

In [ ]:
nwt.coin_gecko.add_value_and_price(balances)

In [ ]:
nwt.utils.save_data(balances, yieldwatch)

## Historic overview

ATH is a personal ATH and 24h and 1w price change accuracy depends on sampling frequency of this notebook.

In [ ]:
import net_worth_tracker as nwt

import pandas as pd

datas = nwt.utils.load_data()
df = nwt.utils.datas_to_df(datas)

overview = nwt.utils.overview_df(df)
total_ath_value = overview["ATH value (€)"].sum(skipna=True)
total_atl_value = overview["ATL value (€)"].sum(skipna=True)
total_value = overview["value"].sum(skipna=True)
print(f"Current networth in crypto:\n€{total_value:.2f}\n")
print(f"The total ATL/ATH value is\n€{total_atl_value:.2f}/€{total_ath_value:.2f}\n")

In [ ]:
pd.set_option("display.float_format", lambda x: "%.5f" % x)
nwt.utils.styled_overview_df(df)

# Plots

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15, 15))
last = (
    df[df.date == df.date.max()]
    .set_index("symbol")
    .value.dropna()
    .sort_values(ascending=False)
)
patches, texts, _ = ax.pie(last, labels=last.index, autopct="%1.1f%%")
factor = 100 / last.sum()
legend_labels = [
    f"{coin} - {factor*amount:1.2f}% - €{amount:.2f}" for coin, amount in last.items()
]

ax.axis("equal")
plt.tight_layout()
plt.legend(
    patches,
    legend_labels,
    loc="upper left",
    bbox_to_anchor=(-0.25, 1.0),
    fontsize=12,
)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

cols = [col for col in df.columns if col.startswith("value_in")]
value_in = df[df.date == df.date.max()][cols].sum(skipna=True).sort_values(ascending=False)
value_in.index = [col.replace("value_in_", "") for col in value_in.index]

fig, ax = plt.subplots(figsize=(8, 8))
patches, texts, _ = ax.pie(value_in, labels=value_in.index, autopct="%1.1f%%")
factor = 100 / value_in.sum()
legend_labels = [
    f"{coin} - {factor*amount:1.2f}% - €{amount:.2f}" for coin, amount in value_in.items()
]

ax.axis("equal")
plt.tight_layout()
plt.legend(
    patches,
    legend_labels,
    loc="upper left",
    bbox_to_anchor=(-0.25, 1.0),
    fontsize=16,
)
plt.show()

In [ ]:
cols = [col for col in df.columns if col.startswith("value_in")]
value_ratios = df.groupby('date').sum()[cols]
value_ratios.columns = [col.replace("value_in_", "") for col in value_ratios.columns]
value_ratios.plot(xlabel="Date", ylabel="Amount (€)", ylim=(0, None), title="Value per category")
plt.show()

## Total value over time

In [ ]:
def get_title(series):
    coin = series.name
    pct_change = 100 * (series.iloc[-1] - series.iloc[0]) / series.iloc[0]
    amount_change = series.iloc[-1] - series.iloc[0]
    return (
        f"{coin} value over time, {pct_change:.1f}%, "
        f"€{amount_change:.2f} change since the start"
    )

total = df.groupby("date").sum()['value']
total.name = "total"
total.plot(
    title=get_title(total),
    xlabel="Date",
    ylabel="Value (€)",
)
plt.show()

## Value over time for current top 15 holdings

In [ ]:
total = df.groupby(["date", "symbol"]).sum()[["value", "amount"]]
total = total.reset_index().set_index("date")
last = total[total.index == total.index.max()]
top_n = 15
biggest_holdings = last.sort_values("value", ascending=False).iloc[:top_n]

for coin in biggest_holdings.symbol:
    sel = total[total.symbol == coin]
    bal = sel.amount
    eur = sel.value
    fig0, ax0 = plt.subplots()
    ax1 = ax0.twinx()
    ax1.spines["right"].set_edgecolor("C1")
    ax1.tick_params(axis="y", colors="C1")
    ax1.set_ylabel(f"Amount in {coin}")
    mid, min_, max_ = bal.mean(), bal.min(), bal.max()
    ax1.set_ylim(min(mid * 0.9, min_), max(mid * 1.1, max_))
    eur.plot(ax=ax0, xlabel="Date", ylabel="Value (€)", title=get_title(eur))
    bal.plot(ax=ax1, c="C1")
    plt.show()

# DeFi on BSC

Top 5 fastest growing holdings in smart contracts.

In [ ]:
df_bsc = nwt.utils.get_df_wallet("bsc", datas)

pct_change = 100 * (df_bsc.iloc[-1] - df_bsc.iloc[0]) / df_bsc.iloc[0]
amount_change = df_bsc.iloc[-1] - df_bsc.iloc[0]
best_5 = pct_change.sort_values(ascending=False)[:5]

for coin, pct_change in best_5.iteritems():
    df_bsc[coin].plot(
        xlabel="Date",
        ylabel=f"Amount ({coin})",
        title=f"{coin} amount, {pct_change:.1f}% change since beginning",
    )
    plt.show()